# Tour de Pologne Top 10 finishers prediction part 1 of 3 (data mining)<a class='tocSkip'>
Main idea is to predict who will finish in Top 10 position after each stage.<br>
Whole process is going to be updated and described thouroughly very soon, please stay tuned.

**Next notebooks:** <br>
[Part 2: Data stats and visuals](tdp_2_3_data_statistics_visualisation.ipynb)<br>
[Part 3: Data  modeling](tdp_3_3_data_modeling.ipynb)

In [1]:
import numpy as np
import pandas as pd

import re

import requests
from bs4 import BeautifulSoup

In [2]:
# Base URL for web scraping
url_base = 'http://firstcycling.com/race.php?r=19&y='

In [3]:
# Defining year range for data of interest
# Before the year of 2014 there is lack of general classification positions (GC) and time (GC Time) information
tour_year_start = 2014
tour_year_stop = 2018
tour_years = list(range(tour_year_start, tour_year_stop+1))
tour_years

[2014, 2015, 2016, 2017, 2018]

<br><br><br>
# Tour results by stages
<br>

In [4]:
# List of tour years for analysis
url_tour_results = []
for year in tour_years:
    url_tour_results.append(url_base + str(year))
# url_tour_results

In [5]:
# List of all stages in the above chosen tours
url_stages_results = []
for stage in url_tour_results:
    for s in range(1,8):
        url_stages_results.append(stage + '&k=etapper&e=0' + str(s))
# url_stages_results

### Generating dictionaries of biker and team name with corresponding system number  that are needed for further steps

In [6]:
biker_dict_stage = {}
team_dict_stage = {}
i = tour_year_start - 1
for stage in [stage for stage in url_stages_results if 'e=01' in stage]:
    # Preparing dataframe with all racer names
    dict_prep_ = pd.read_html(stage)
    dict_prep = dict_prep_[0]
    dict_prep['Name'] = dict_prep['Name'].str[0] + '.' + dict_prep['Name'].str.split(' ', 1, expand=True)[1]
    
    # Scraping system numbers
    r = requests.get(stage)
    soup = BeautifulSoup(r.text,'html.parser')
    system_numbers = soup.find('table')

    # Schema as `rider.php?r=2165` (2-5 digits with margin)
    biker_system_nr = pd.DataFrame(re.findall('rider.php\?r=(\d{2,6})', str(system_numbers)), columns=['biker_sys_nr'])
    # Schema as `team.php?l=8433` (3-4 digits with margin)
    team_system_nr = pd.DataFrame(re.findall('team.php\?l=(\d{3,5})', str(system_numbers)), columns=['team_sys_nr'])
    
    # Merging team and rider names with their system numbers 
    biker_dict_stage_ = dict(zip(dict_prep['Name'], biker_system_nr['biker_sys_nr']))
    biker_dict_stage.update(biker_dict_stage_)
    team_dict_stage_ = dict(zip(dict_prep['Team'], team_system_nr['team_sys_nr']))
    team_dict_stage.update(team_dict_stage_)
    i += 1
    print('Created dictionary of bikers participating in year ' + str(i))

Created dictionary of bikers participating in 2014
Created dictionary of bikers participating in 2015
Created dictionary of bikers participating in 2016
Created dictionary of bikers participating in 2017
Created dictionary of bikers participating in 2018


In [7]:
# Overall number of bikers and teams
len(biker_dict_stage), len(team_dict_stage)

(513, 69)

### Defining datetime functions

In [8]:
# Adding leading zeros for further splitting based on ':' character
def zeros_to_sec(df, col1, col2=False):
    df[col1] = np.where(df[col1].str.len() == 2, '00:00:' + df[col1],'00:' + df[col1]) 
    df[col1] = df[col1].where(df[col1].str.len() != 11, df[col1].str.slice(3,11))
    
    if col2:
        df[col2] = np.where(df[col2].str.len() == 2, '00:00:' + df[col2],'00:' + df[col2])
        df[col2] = df[col2].where(df[col2].str.len() != 11, df[col2].str.slice(3,11))

In [9]:
# Changing overall time to seconds
def time_to_sec(df, col1, col2=False):
    df[col1] = [(int(t[0])*3600 + int(t[1])*60 + int(t[2])) for t in df[col1].str.split(':')]
    df[col1] = df[col1] + df[col1].max()
    df.at[0, [col1]] = df[col1].max() / 2

    if col2:
        df[col2] = [(int(t[0])*3600 + int(t[1])*60 + int(t[2])) for t in df[col2].str.split(':')]
        df[col2] = df[col2] + df[col2].max()
        df.at[df[col2].idxmax(), [col2]] = df[col2].max() / 2

### Web scraping tour's stage-by-stage info for adding new columns with extended statistics. Output: `stages`

In [10]:
# Using URLs with all-year stages results
# url_stages_results[:]

In [11]:
stages_ = []

for year in tour_years:
    i = 1
    for stage_ in [stage_ for stage_ in url_stages_results if 'y='+str(year) in stage_]:
        try:
            stages__ = pd.read_html(stage_)
            stages__ = stages__[0]
            stages__['year'] = year
            # Limiting number of stages in each year to 7
            if i <= 7:
                stages__['stage'] = i
            else:
                i = 1        
            stages__.columns = ['position','to_drop','name','age','team','time_in_s','gc','gc_time_in_s','year','stage']
            stages__['age'] = year - stages__['age']
            stages__.replace(to_replace=r'\+ ', value='', regex=True, inplace=True)
            stages__.replace(to_replace=[np.NaN], value='00', inplace=True)
            
            zeros_to_sec(stages__, 'time_in_s','gc_time_in_s')
            time_to_sec(stages__, 'time_in_s','gc_time_in_s')
            
            # Setting DNF and DNS times to some big number
            try:
                stages__['time_in_s'].where((stages__['position'] != 'DNF') & (stages__['position'] != 'DNS'),
                                            999999, inplace=True)
                stages__['gc_time_in_s'].where((stages__['position'] != 'DNF') & (stages__['position'] != 'DNS'),
                                               999999, inplace=True)
            except TypeError:
                pass
            stages__.replace(to_replace=['DNF', 'DNS'], value=999, inplace=True)
            stages__['position'] = stages__['position'].astype('int')
            
            stages_.append(stages__)
            print('Scraped tour of', year, '/ Stage', i)
            i += 1
        except ValueError:
            print('There is no data for this stage. Skipping...')
            i += 1
            pass

Scraped tour of 2014 / Stage 1


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2014 / Stage 2


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2014 / Stage 3


...

Scraped tour of 2017 / Stage 2
Scraped tour of 2017 / Stage 3
Scraped tour of 2017 / Stage 4
Scraped tour of 2017 / Stage 5
Scraped tour of 2017 / Stage 6
Scraped tour of 2017 / Stage 7


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2018 / Stage 1


C:\Users\cyberdeck\Anaconda3\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Scraped tour of 2018 / Stage 2
Scraped tour of 2018 / Stage 3
Scraped tour of 2018 / Stage 4
Scraped tour of 2018 / Stage 5
Scraped tour of 2018 / Stage 6
Scraped tour of 2018 / Stage 7


In [12]:
# Merging above stages into DataFrame
stages = pd.concat(stages_, axis=0)
stages.reset_index(drop=True, inplace=True)

# Dropping columns with no information
stages.drop('to_drop', axis=1, inplace=True)

# Shortening biker names into 'initial.surname' form
stages['name'] = stages['name'].str[0] + '.' + stages['name'].str.split(' ', 1, expand=True)[1]

# Creating Biker and Team system numbers
stages['biker_sys_nr'] = stages['name'].map(biker_dict_stage)
stages['team_sys_nr'] = stages['team'].map(team_dict_stage)

# Rearranging columns
stages = stages[['name','biker_sys_nr','age','team','team_sys_nr','time_in_s','gc','gc_time_in_s','year','stage','position']]

In [13]:
stages.head()

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,position
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,1
1,R.Maikin,14004,24,RusVelo,3482,20870.0,2,20864.0,2014,1,2
2,M.Mori,492,34,Lampre - Merida,6418,20870.0,3,20866.0,2014,1,3
3,G.Boivin,1937,25,Cannondale Pro Cycling,3481,20870.0,6,20870.0,2014,1,4
4,M.Haller,6246,23,Team Katusha,8441,20870.0,7,20870.0,2014,1,5


### Creating columns with different statistics for each biker. Output: `biker_by_stage` 

In [14]:
'''
'n_tours'            - number of tours started in
'n_stages'           - number of stages started in
'avg_year'          - mean year from all tours
'best_pos_ovr'       - best position from all stages
'worst_pos_ovr'      - worst position from all stages
'avg_pos_ovr'        - average position from all stages
'n_top10_ovr'        - number of top10 positions in all stages 
'perc_win_ovr'       - percentage of winning throughout all stages
'perc_top10_ovr'     - percentage of being in top10 in all stages
'avg_speed_ovr'      - average speed throughout all stages
'best_pos_l...'      - best position from last 1-3 stages
'wors_pos_l...'      - etc...
'avg_pos_l...'
'n_top10_l...'
'perc_win_l...'
'perc_top10_l...'
'avg_speed_l...'
'''

biker_by_stage_ = pd.DataFrame()

# Number of tours and stages ridden by Biker
biker_by_stage_['n_tours'] = stages.groupby(['name'])['year'].nunique()
biker_by_stage_['n_stages'] = stages.groupby(['name'])['stage'].count()
biker_by_stage_['avg_year'] = stages.groupby('name')['year'].mean()

# Best, average and worst tour stats
min_max_ovr = stages.groupby('name')[['position','time_in_s','gc','gc_time_in_s']].agg(['min','mean','max'])

# Number of Top 10 positions
top10 = pd.DataFrame(stages.groupby(stages[stages['position'] <= 10]['name'])['position'].count())

# Number of won stages
winner_ = pd.DataFrame(stages.groupby(stages[stages['position'] == 1]['name'])['position'].count())

# Defining the same data as above but only for last 3 stages
last_3_stages = stages.sort_values(['name','year','stage']).groupby('name').tail(3)
min_max_avg_l3 = last_3_stages.groupby('name')[['position','time_in_s','gc','gc_time_in_s']].agg(['min','mean','max'])
top10_l3 = pd.DataFrame(last_3_stages[last_3_stages['position'] <= 10]['name'].value_counts())
winner_l3 = pd.DataFrame(last_3_stages.groupby(last_3_stages[last_3_stages['position'] == 1]['name'])['position'].count())

# Merging all info into one DataFrame
biker_by_stage_ = pd.concat([biker_by_stage_, min_max_ovr, top10, winner_, min_max_avg_l3, top10_l3, winner_l3], axis=1, sort=False)
biker_by_stage_ = biker_by_stage_.replace(np.NaN, 0)
biker_by_stage_.columns = ['n_tours','n_stages','avg_year','best_pos_ovr','avg_pos_ovr','worst_pos_ovr','best_time_ovr','avg_time_ovr','worst_time_ovr',
                         'best_gc_ovr','avg_gc_ovr','worst_gc_ovr','best_gc_time_ovr','avg_gc_time_ovr','worst_gc_time_ovr','n_top10_ovr','n_win_ovr',
                         'best_pos_ovr_l3','avg_pos_ovr_l3','worst_pos_ovr_l3','best_time_ovr_l3','avg_time_ovr_l3','worst_time_ovr_l3',
                         'best_gc_ovr_l3','avg_gc_ovr_l3','worst_gc_ovr_l3','best_gc_time_ovr_l3','avg_gc_time_ovr_l3','worst_gc_time_ovr_l3',
                         'n_top10_l3','n_win_l3']

# Adding columns of Top 10 and winnig stage percetage
biker_by_stage_['perc_top10_ovr'] = biker_by_stage_['n_top10_ovr'] / biker_by_stage_['n_stages']
biker_by_stage_['perc_win_ovr'] = biker_by_stage_['n_win_ovr'] / biker_by_stage_['n_stages']
# biker_by_stage_

In [16]:
# Merging above results into DataFrame
# Each row is an information about each stage ridden by the biker together with his personal info,
# counted statistics and historical data
biker_by_stage = pd.merge(stages, biker_by_stage_, left_on='name', right_on=biker_by_stage_.index)
biker_by_stage.head()

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,...,best_gc_ovr_l3,avg_gc_ovr_l3,worst_gc_ovr_l3,best_gc_time_ovr_l3,avg_gc_time_ovr_l3,worst_gc_time_ovr_l3,n_top10_l3,n_win_l3,perc_top10_ovr,perc_win_ovr
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
1,Y.Hutarovich,100,31,AG2R La Mondiale,10072,19455.0,2,40315.0,2014,2,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
2,Y.Hutarovich,100,31,AG2R La Mondiale,10072,13167.0,2,53482.0,2014,3,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
3,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20609.0,3,74091.0,2014,4,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
4,Y.Hutarovich,100,31,AG2R La Mondiale,10072,17350.0,98,91441.0,2014,5,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857


### Manually setting captured jerseys

In [18]:
# TBD

<br><br><br>
# Stages summary
Overall summary of all tours' stages
<br>

In [19]:
# Using 'url_tour_results' for creating stages summary data
url_stages_summary = []
for tour in url_tour_results:
    url_stages_summary.append(tour + '&k=etapper')
# url_stages_summary

In [20]:
# Web scraping stages summary data
stages_summary_ = []
for year in tour_years:
    for stage in [stage for stage in url_stages_summary if 'y='+str(year) in stage]:
        stages_summary__ = pd.read_html(stage)
        stages_summary__ = pd.DataFrame(stages_summary__[0])
        stages_summary__['year'] = year
        stages_summary_.append(stages_summary__)
stages_summary = pd.concat(stages_summary_, axis=0)
stages_summary.head(8)

,Unnamed: 0,Unnamed: 1,Date,Km,Unnamed: 4,Finish,Winner,Leader,Points,Mountains,Sprint,year
0,1,NaN,3.Aug,226.0,Results,Bygdoszcz,Y.Hutarovich,Y.Hutarovich,Y.Hutarovich,M.Paterski,J.Engoulvent,2014
1,2,NaN,4.Aug,226.0,Results,Warszawa,P.Vakoc,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
2,3,NaN,5.Aug,174.0,Results,Rzeszów,T.Bos,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
3,4,NaN,6.Aug,236.0,Results,Katowice,J.Van Genechten,P.Vakoc,Y.Hutarovich,M.Taciak,M.Krizek,2014
4,5,NaN,7.Aug,190.0,Results,Strbskie Pleso,R.Majka,P.Vakoc,Y.Hutarovich,M.Paterski,M.Krizek,2014
5,6,NaN,8.Aug,174.0,Results,Bukowina Tatrzanska,R.Majka,R.Majka,Y.Hutarovich,M.Paterski,M.Krizek,2014
6,7,NaN,9.Aug,25.0,Results,Krakow,K.Vandewalle,R.Majka,Y.Hutarovich,M.Paterski,M.Krizek,2014
0,1,NaN,2.Aug,122.0,Results,Warszawa,M.Kittel,M.Kittel,M.Kittel,A.Kurek,M.Mohoric,2015


### Specifying stage type by its webpage icon

In [21]:
# Since icons are named in Norwegian, I create Norwegian-English dictionary of stage types
# stage_names_nor = list(set(stage_type))
stage_names_nor = ['Flatt','Smaakupert','Smaakupert-MF','Fjell','Fjell-MF', 'Tempo']
stage_names_en = ['flat','hilly','hilly-mf','mountain','mountain-mf','itt']
stage_names = dict(zip(stage_names_nor, stage_names_en))
stage_type_recode = dict(zip(stage_names_nor, list(range(1,7)))) 

stage_names

{'Fjell': 'mountain',
 'Fjell-MF': 'mountain-mf',
 'Flatt': 'flat',
 'Smaakupert': 'hilly',
 'Smaakupert-MF': 'hilly-mf',
 'Tempo': 'itt'}

In [22]:
# Historically (but only before 2014 year), there were also two more stage types,
# but here I skip them as they are no longer valid.
# Bakketempo : mountain-itt; Ukjent : '?' (question mark); 

# Creating a list od all stage types thorought all analyzed tours
stage_type = []
for stage in url_stages_summary:
    r = requests.get(stage)
    soup = BeautifulSoup(r.text,'html.parser')

    # Selecting all file names used in naming stage type
    stage_soup = soup.find_all('img')
    stage_img = re.findall('([A-Z][a-z].*?)\.gif', str(stage_soup))
    
    # Selecting only names correspoding to stage types 
    for name_ in stage_img:
        if name_ in stage_names_nor:
            stage_type.append(name_)
            
# stage_type

In [23]:
stages_summary.head()

,Unnamed: 0,Unnamed: 1,Date,Km,Unnamed: 4,Finish,Winner,Leader,Points,Mountains,Sprint,year
0,1,NaN,3.Aug,226.0,Results,Bygdoszcz,Y.Hutarovich,Y.Hutarovich,Y.Hutarovich,M.Paterski,J.Engoulvent,2014
1,2,NaN,4.Aug,226.0,Results,Warszawa,P.Vakoc,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
2,3,NaN,5.Aug,174.0,Results,Rzeszów,T.Bos,P.Vakoc,Y.Hutarovich,M.Paterski,P.Vakoc,2014
3,4,NaN,6.Aug,236.0,Results,Katowice,J.Van Genechten,P.Vakoc,Y.Hutarovich,M.Taciak,M.Krizek,2014
4,5,NaN,7.Aug,190.0,Results,Strbskie Pleso,R.Majka,P.Vakoc,Y.Hutarovich,M.Paterski,M.Krizek,2014


### Columns rearranging and cleaning

In [24]:
# Assigning stage type to appropriate column
stages_summary['Unnamed: 1'] = stage_type

# Recoding categorical stage type names to corresponding numerics and rearranging columns
stages_summary.columns = ['stage','stage_type','date','dist','to_drop','finish','stage_winner','stage_leader','points_leader','mountains_leader','sprint_leader','year']
stages_summary['stage_type_code'] = stages_summary['stage_type'].map(stage_type_recode)
stages_summary = stages_summary[['date','year','stage','stage_type','stage_type_code','dist','to_drop','finish','stage_winner','stage_leader','points_leader','mountains_leader','sprint_leader']]

# Mapping English names to Norwegian descriptions
stages_summary['stage_type'] = stages_summary['stage_type'].map(stage_names)

stages_summary.drop('to_drop', axis=1, inplace=True)

# Selecting only appropriate columns
stages_summary = stages_summary.loc[:, 'date':'finish'].reset_index(drop=True)
stages_summary.head()

,date,year,stage,stage_type,stage_type_code,dist,finish
0,3.Aug,2014,1,flat,1,226.0,Bygdoszcz
1,4.Aug,2014,2,flat,1,226.0,Warszawa
2,5.Aug,2014,3,flat,1,174.0,Rzeszów
3,6.Aug,2014,4,flat,1,236.0,Katowice
4,7.Aug,2014,5,hilly,2,190.0,Strbskie Pleso


### Parsing stage date into appropriate format

In [25]:
from datetime import timedelta

In [26]:
stages_summary['date'] = stages_summary['date'] + '.' + stages_summary['year'].astype('str')

In [27]:
stages_summary['date'] = pd.to_datetime(stages_summary['date'], format='%d.%b.%Y')
stages_summary['date'][:3]

0   2014-08-03
1   2014-08-04
2   2014-08-05
Name: date, dtype: datetime64[ns]

### Weather info

In [28]:
# Usefull information taken into consideration that can have an impact on the stage results are:
# Precipitation(0/1; TBD),Temp(float), Cloud overcast and a few others
# Data scraped from: https://dane.imgw.pl/data/dane_pomiarowo_obserwacyjne/

'''
Technical names for different weather parameters with numbers indicating lenght of the value:

Station code                         9
Station name                        30
Year                                 4
Month                                2
Day                                  2
Hour                                 2
Air temperature                    6/1
TEMP measurement status              1
Wetted thermometer temperature     6/1
TTZW measurement status              1
Ice indicator                        1
Ventilation indicator                1
Relative humidity                    5
WLGW measurement indicator           1
Wind direction code                  3
DKDK measurement indicator           1
Wind speed                           5
FWR measurement indicator            1
Overall overcast                     5
ZOGK measurement indicator           1
Visibility                           5
WID measurement indicator            1

Status "8" means lack of measurement
Overcast:
        - 0 - 10 scale up to 31.12.1988
        - 0 – 8 scale up to 01.01.1989
'''

names = ['station_code','station_name','year','month','day','hour',
         'temp','temp_status','wet_temp','ttzw_status','ice_ind','vent_ind',
         'rel_humid','wlgw_status','wind_dir','dkdk_status','wind_speed','fwr_status',
         'clouds_ovrl','zogk_status','visibility','wid_status']

# NOTE: There are no available data for the proper month of 2018. Will use them upon availability. 
weather_08_14 = pd.read_csv('data/weather_k_t_08_2014.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_08_15 = pd.read_csv('data/weather_k_t_08_2015.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_07_16 = pd.read_csv('data/weather_k_t_07_2016.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_07_17 = pd.read_csv('data/weather_k_t_07_2017.csv', encoding = 'ISO-8859-2', header=None, names=names)
weather_08_17 = pd.read_csv('data/weather_k_t_08_2017.csv', encoding = 'ISO-8859-2', header=None, names=names)

# Using data from earlier month
weather_08_18 = pd.read_csv('data/weather_k_t_07_2018.csv', encoding = 'ISO-8859-2', header=None, names=names)

In [29]:
# Reassigning proper values for comparing weather date with the stage date
# Once appropriate data will be available this step is redundant
weather_08_18['month'] = 8

In [30]:
# Gathering all above weather information into one DataFrame
weather_info = pd.concat([weather_08_14, weather_08_15,weather_07_16,
                          weather_07_17, weather_08_17, weather_08_18], axis=0)
# Choosing time of measurement as 12 o'clock
weather_info = weather_info[weather_info.hour == 12].reset_index(drop=True)
weather_info.head()

,station_code,station_name,year,month,day,hour,temp,temp_status,wet_temp,ttzw_status,...,rel_humid,wlgw_status,wind_dir,dkdk_status,wind_speed,fwr_status,clouds_ovrl,zogk_status,visibility,wid_status
0,249180010,PSZCZYNA,2014,8,1,12,22.4,NaN,0.0,8.0,...,0,8.0,C,NaN,0,NaN,5,NaN,6,NaN
1,249180010,PSZCZYNA,2014,8,2,12,29.5,NaN,0.0,8.0,...,0,8.0,E,NaN,2,NaN,0,NaN,8,NaN
2,249180010,PSZCZYNA,2014,8,3,12,29.4,NaN,0.0,8.0,...,0,8.0,SSE,NaN,2,NaN,1,NaN,8,NaN
3,249180010,PSZCZYNA,2014,8,4,12,20.2,NaN,0.0,8.0,...,0,8.0,WSW,NaN,2,NaN,4,NaN,8,NaN
4,249180010,PSZCZYNA,2014,8,5,12,22.4,NaN,0.0,8.0,...,0,8.0,SE,NaN,1,NaN,3,NaN,6,NaN


In [31]:
# Merging all weather date columns to compare them later with stage dates
weather_info['date'] = weather_info['year'].astype('str') + '.' +\
                       weather_info['month'].astype('str') + '.' +\
                       weather_info['day'].astype('str')
weather_info['date'] = pd.to_datetime(weather_info['date'], format='%Y.%m.%d')
weather_info.head()

,station_code,station_name,year,month,day,hour,temp,temp_status,wet_temp,ttzw_status,...,wlgw_status,wind_dir,dkdk_status,wind_speed,fwr_status,clouds_ovrl,zogk_status,visibility,wid_status,date
0,249180010,PSZCZYNA,2014,8,1,12,22.4,NaN,0.0,8.0,...,8.0,C,NaN,0,NaN,5,NaN,6,NaN,2014-08-01
1,249180010,PSZCZYNA,2014,8,2,12,29.5,NaN,0.0,8.0,...,8.0,E,NaN,2,NaN,0,NaN,8,NaN,2014-08-02
2,249180010,PSZCZYNA,2014,8,3,12,29.4,NaN,0.0,8.0,...,8.0,SSE,NaN,2,NaN,1,NaN,8,NaN,2014-08-03
3,249180010,PSZCZYNA,2014,8,4,12,20.2,NaN,0.0,8.0,...,8.0,WSW,NaN,2,NaN,4,NaN,8,NaN,2014-08-04
4,249180010,PSZCZYNA,2014,8,5,12,22.4,NaN,0.0,8.0,...,8.0,SE,NaN,1,NaN,3,NaN,6,NaN,2014-08-05


In [32]:
# Based on known stage location I manually chose corresponding meteo stations 
# This is to be further tweaked for choosing more accurate (closer) meteo st. locations

weather_stages = pd.DataFrame()
for year in tour_years:
    # Stage 1
    st1_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('KRAK')].iloc[0] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 2
    st2_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('DRONI')].iloc[1] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 3
    st3_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('BRENNA')].iloc[2] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 4
    st4_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('DRONI')].iloc[3] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 5
    st5_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('CHORZ')].iloc[4] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 6
    st6_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('LIMA')].iloc[5] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    # Stage 7
    st7_weather = weather_info[weather_info.date.isin(stages_summary.date) \
                    & (weather_info.year == year) \
                    & weather_info.station_name.str.contains('BUKOW')].iloc[6] \
                    [['temp','rel_humid','wind_dir','wind_speed','clouds_ovrl','visibility']]
    weather_stages = weather_stages.append([st1_weather, st2_weather, st3_weather, st4_weather,
                           st5_weather, st6_weather, st7_weather]).reset_index(drop=True)

In [33]:
# Chosen weather info for all tour stages summaries
weather_stages.head(8)

,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility
0,31.5,38,ESE,2,6,9
1,26.6,61,NW,4,5,7
2,22.5,65,SSE,1,4,6
3,22.7,52,N,3,4,8
4,25.5,66,ENE,2,1,8
5,21.4,73,NW,4,7,8
6,22.9,0,NaN,0,5,7
7,27.1,38,SSE,1,7,9


### Joining stages summary and weather info dataframes. The output is `stages_summary`

In [34]:
stages_summary = pd.concat([stages_summary, weather_stages], axis=1)
stages_summary.head(8)

,date,year,stage,stage_type,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility
0,2014-08-03,2014,1,flat,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9
1,2014-08-04,2014,2,flat,1,226.0,Warszawa,26.6,61,NW,4,5,7
2,2014-08-05,2014,3,flat,1,174.0,Rzeszów,22.5,65,SSE,1,4,6
3,2014-08-06,2014,4,flat,1,236.0,Katowice,22.7,52,N,3,4,8
4,2014-08-07,2014,5,hilly,2,190.0,Strbskie Pleso,25.5,66,ENE,2,1,8
5,2014-08-08,2014,6,hilly,2,174.0,Bukowina Tatrzanska,21.4,73,NW,4,7,8
6,2014-08-09,2014,7,itt,6,25.0,Krakow,22.9,0,NaN,0,5,7
7,2015-08-02,2015,1,flat,1,122.0,Warszawa,27.1,38,SSE,1,7,9


<br><br><br>
# Bikers statistics and overall description scraping
<br>

In [35]:
# Overall bikers list
biker_base_url = 'http://firstcycling.com/rider.php?r='

bikers_ranking_urls = []
for k,v in biker_dict_stage.items():
    bikers_ranking_urls.append(biker_base_url + v)
print('Number of all bikers:', len(bikers_ranking_urls))
# pd.DataFrame(bikers_ranking_urls)[0]

Number of all bikers: 513


### Scraping bikers stats

**(NOTICE: This operation takes around 16 minutes)**

In [36]:
STOP

NameError: name 'STOP' is not defined

In [65]:
bikers_stats_ = pd.DataFrame()
i = -1
for url_ in bikers_ranking_urls:

    # Stats scraping with the season-by-season separation
    r = requests.get(url_)
    table_ = pd.read_html(r.text)   
    # Settig 'Season' column as an index for dataframe transposition
    biker_rank = table_[0].loc[:4,:].set_index('Season')
    biker_rank.columns = ['div','team','fc_rank','to_drop','uci_rank','to_drop','race_days','wins','year_km']
    biker_rank.drop(['to_drop'], axis=1, inplace=True)
    # Dataframe transposition
    biker_rank = biker_rank.stack().to_frame().T
    # Renaming stacked column names
    biker_rank.columns = ['{}_{}'.format(*c) for c in biker_rank.columns]  
    # Removing duplicate rows
    biker_rank = biker_rank.loc[:,~biker_rank.columns.duplicated()]
    
    # Physical stats, nationality, won jerseys, all-time victories and followers number extraction
    soup = BeautifulSoup(r.text,'html.parser')
    stats = soup.find_all('div', class_='back')
    
    # Not all bikers have all data available
    nation = re.findall('nat=.*?>(\w.*?)<|$', str(stats))[0]
    try:
        height = re.findall('Height.*?(\d{1}\.\d{2})|$', str(stats))[0] 
    except (ValueError, IndexError):
        height = np.NaN
    try:
        wins_search = re.search('Victories.*?<h2', str(stats)).group() 
        total_wins = len(re.findall('\d{4}(,|\))', wins_search))
    except (AttributeError, ValueError, IndexError):
        total_wins = 0
    try:
        youth_search = re.search('Youth.*?<\/p', str(stats)).group()
        youth_jersey = len(re.findall('[( ,](\d{4})', youth_search))
    except (AttributeError, ValueError, IndexError):
        youth_jersey = 0
    try:
        point_search = re.search('Points.*?<\/p', str(stats)).group()
        point_jersey = len(re.findall('[( ,](\d{4})', point_search))
    except (AttributeError, ValueError, IndexError):
        point_jersey = 0
    try:
        mount_search = re.search('Mount.*?<\/p', str(stats)).group()
        mount_jersey = len(re.findall('[( ,](\d{4})', mount_search))
    except (AttributeError, ValueError, IndexError):
        mount_jersey = 0
    try:
        sprint_search = re.search('Sprint.*?<\/p', str(stats)).group()
        sprint_jersey = len(re.findall('[( ,](\d{4})', sprint_search))
    except (AttributeError, ValueError, IndexError):
        sprint_jersey = 0
    try:
        followers_soup = soup.find_all('p', class_='small')
        followers = len(re.findall('ID', str(followers_soup)))
    except (AttributeError, ValueError, IndexError):
        followers = 0
    
    # All separate information merged into one 'stats_table' dataframe
    results_dict = {
        'nation':nation,
        'height':height,
        'total_wins':total_wins,
        'youth_jerseys':youth_jersey,
        'point_jerseys':point_jersey,
        'mount_jerseys':mount_jersey,
        'sprint_jerseys':sprint_jersey,
        'followers':followers
    }
    stats_table = pd.DataFrame(data=results_dict, index=[0])
    
    # Merging dataframes
    biker_rank = pd.concat([biker_rank, stats_table], axis=1)
    i += 1
    print('Scraped biker no.', i, '/', len(bikers_ranking_urls),nation, height, total_wins, youth_jersey, point_jersey,
          mount_jersey, sprint_jersey, followers)
    bikers_stats_ = bikers_stats_.append(biker_rank, sort=False)

Scraped biker no. 0 / 513 Belarus 1.79 8 0 6 0 0 0
Scraped biker no. 1 / 513 Russia  0 0 2 0 0 2
Scraped biker no. 2 / 513 Italy 1.72 4 0 0 0 0 0
Scraped biker no. 3 / 513 Canada 1.80 4 0 1 0 0 3
Scraped biker no. 4 / 513 Austria 1.78 2 1 0 1 0 3
Scraped biker no. 5 / 513 Belgium 1.90 4 1 1 0 0 0
Scraped biker no. 6 / 513 Poland  9 0 1 0 0 2
Scraped biker no. 7 / 513 Denmark 1.75 6 0 0 0 2 0
Scraped biker no. 8 / 513 Italy 1.81 3 1 0 0 0 7
Scraped biker no. 9 / 513 Italy 1.74 8 1 1 0 0 2
Scraped biker no. 10 / 513 Poland  3 0 2 0 2 1
Scraped biker no. 11 / 513 Italy 1.70 7 0 0 0 0 6
Scraped biker no. 12 / 513 Italy 1.75 4 1 1 2 0 1
Scraped biker no. 13 / 513 Netherlands 1.90 12 0 2 0 0 1
Scraped biker no. 14 / 513 Russia  8 1 0 0 0 0
Scraped biker no. 15 / 513 Germany 1.80 2 0 0 0 0 0
Scraped biker no. 16 / 513 Costa Rica 1.80 1 0 0 0 0 5
Scraped biker no. 17 / 513 Poland 1.82 4 0 0 0 0 3
Scraped biker no. 18 / 513 France 1.74 5 0 0 1 0 2
Scraped biker no. 19 / 513 Poland  5 1 0 1 0 0


Scraped biker no. 159 / 513 Great Britain 1.81 1 0 0 1 0 0
Scraped biker no. 160 / 513 USA 1.72 6 0 1 0 0 0
Scraped biker no. 161 / 513 Australia 1.80 5 0 0 1 0 0
Scraped biker no. 162 / 513 France 1.78 0 0 0 0 0 0
Scraped biker no. 163 / 513 Slovenia 1.77 7 3 0 0 0 1
Scraped biker no. 164 / 513 Italy 1.72 13 0 1 0 0 5
Scraped biker no. 165 / 513 Italy 1.72 13 0 1 0 0 5
Scraped biker no. 166 / 513 Germany 1.88 29 2 10 0 0 13
Scraped biker no. 167 / 513 Australia  17 1 5 0 0 9
Scraped biker no. 168 / 513 Netherlands  2 1 0 0 0 1
Scraped biker no. 169 / 513 France 1.79 1 0 0 0 0 0
Scraped biker no. 170 / 513 Belgium 1.80 10 0 1 0 0 2
Scraped biker no. 171 / 513 Italy 1.70 18 0 5 0 0 3
Scraped biker no. 172 / 513 France  1 0 0 0 0 1
Scraped biker no. 173 / 513 Poland 1.76 27 3 4 0 0 17
Scraped biker no. 174 / 513 Norway  8 1 0 0 0 6
Scraped biker no. 175 / 513 Italy 1.84 22 1 6 0 0 6
Scraped biker no. 176 / 513 Croatia 1.79 2 3 0 2 0 1
Scraped biker no. 177 / 513 Germany 1.89 6 0 2 0 2 1


Scraped biker no. 317 / 513 Italy 1.75 28 0 2 1 0 2
Scraped biker no. 318 / 513 Germany  4 0 1 1 0 2
Scraped biker no. 319 / 513 New Zealand  3 1 0 1 0 0
Scraped biker no. 320 / 513 USA  1 0 0 0 0 1
Scraped biker no. 321 / 513 Spain 1.79 1 0 0 1 0 2
Scraped biker no. 322 / 513 Great Britain  1 0 0 0 0 0
Scraped biker no. 323 / 513 Poland  6 0 1 2 0 1
Scraped biker no. 324 / 513 Spain  0 0 0 0 0 0
Scraped biker no. 325 / 513 Portugal 1.72 1 0 0 1 2 2
Scraped biker no. 326 / 513 Belgium 1.82 22 0 0 1 0 3
Scraped biker no. 327 / 513 Lithuania  6 1 0 1 0 2
Scraped biker no. 328 / 513 France  0 0 0 0 0 0
Scraped biker no. 329 / 513 Italy  20 0 0 0 0 2
Scraped biker no. 330 / 513 Italy 1.84 12 0 12 0 0 2
Scraped biker no. 331 / 513 Poland  8 0 0 0 0 0
Scraped biker no. 332 / 513 Poland  6 1 0 1 1 0
Scraped biker no. 333 / 513 Great Britain  2 0 0 1 2 0
Scraped biker no. 334 / 513 Poland  0 0 0 0 0 0
Scraped biker no. 335 / 513 Poland  2 0 0 0 0 1
Scraped biker no. 336 / 513 Ireland  5 0 0 1 

Scraped biker no. 479 / 513 Kazakhstan  2 0 0 1 0 0
Scraped biker no. 480 / 513 Poland  0 0 0 2 0 0
Scraped biker no. 481 / 513 Great Britain  5 1 0 0 0 1
Scraped biker no. 482 / 513 Luxembourg 1.90 3 0 0 1 0 2
Scraped biker no. 483 / 513 Spain 1.73 5 0 5 0 0 4
Scraped biker no. 484 / 513 Denmark 1.90 8 0 0 0 0 1
Scraped biker no. 485 / 513 Belgium  3 1 0 1 0 10
Scraped biker no. 486 / 513 Italy  5 1 0 0 0 4
Scraped biker no. 487 / 513 France 1.85 3 0 0 0 0 0
Scraped biker no. 488 / 513 Argentina  3 1 0 1 0 5
Scraped biker no. 489 / 513 Russia  2 0 0 0 0 1
Scraped biker no. 490 / 513 Australia 1.80 4 4 0 0 0 3
Scraped biker no. 491 / 513 South Africa  0 0 0 0 0 0
Scraped biker no. 492 / 513 Belgium  0 1 0 0 1 4
Scraped biker no. 493 / 513 Australia  7 0 1 0 0 7
Scraped biker no. 494 / 513 Belgium  2 0 0 0 1 2
Scraped biker no. 495 / 513 France  1 1 0 0 0 0
Scraped biker no. 496 / 513 Great Britain  1 1 0 0 0 1
Scraped biker no. 497 / 513 Switzerland 1.72 10 0 2 2 2 3
Scraped biker no. 

In [66]:
# Alphabetically column sorting
bikers_stats_.sort_index(axis=1, inplace=True)

# Assigning biker names from dictionary to the scraped data
bikers_stats_['name'] = [k for k,v in biker_dict_stage.items()][:len(bikers_stats_)]

# Resetting index for further dataframe concatenation
bikers_stats_.reset_index(drop=True, inplace=True)

# Replacing missed height values
bikers_stats_['height'].replace('', np.NaN, inplace=True)

### Merging data into `bikers_stats`

**Choosing only years of 2015 - 2018 due to much missing data in earlier years**

In [67]:
# From the whole scraped historical results dataset (2013-2019) choosing only years 2015-2018 due to many missing values before that timepoint
bikers_stats = pd.concat([bikers_stats_.loc[:,'2015_div':'2018_year_km'], bikers_stats_.iloc[:,-9:]], axis=1)
bikers_stats.head(3)

,2015_div,2015_fc_rank,2015_race_days,2015_team,2015_uci_rank,2015_wins,2015_year_km,2016_div,2016_fc_rank,2016_race_days,...,2018_year_km,followers,height,mount_jerseys,nation,point_jerseys,sprint_jerseys,total_wins,youth_jerseys,name
0,PROF,115,70,Bretagne - Séché Environnement,-,3,11430.9,PROF,430,59,...,NaN,0,1.79,0,Belarus,6,0,8,0,Y.Hutarovich
1,PROF,301,54,RusVelo,-,1,8473.1,PROF,161,63,...,7349.7,2,NaN,0,Russia,2,0,0,0,R.Maikin
2,PRT,640,73,Lampre - Merida,-,0,11960.4,PRT,375,70,...,9924.4,0,1.72,0,Italy,0,0,4,0,M.Mori


### Setting final biker class dataframe. The output is `biker_class`

In [68]:
biker_by_stage.columns

Index(['name', 'biker_sys_nr', 'age', 'team', 'team_sys_nr', 'time_in_s', 'gc',
       'gc_time_in_s', 'year', 'stage', 'position', 'n_tours', 'n_stages',
       'avg_year', 'best_pos_ovr', 'avg_pos_ovr', 'worst_pos_ovr',
       'best_time_ovr', 'avg_time_ovr', 'worst_time_ovr', 'best_gc_ovr',
       'avg_gc_ovr', 'worst_gc_ovr', 'best_gc_time_ovr', 'avg_gc_time_ovr',
       'worst_gc_time_ovr', 'n_top10_ovr', 'n_win_ovr', 'best_pos_ovr_l3',
       'avg_pos_ovr_l3', 'worst_pos_ovr_l3', 'best_time_ovr_l3',
       'avg_time_ovr_l3', 'worst_time_ovr_l3', 'best_gc_ovr_l3',
       'avg_gc_ovr_l3', 'worst_gc_ovr_l3', 'best_gc_time_ovr_l3',
       'avg_gc_time_ovr_l3', 'worst_gc_time_ovr_l3', 'n_top10_l3', 'n_win_l3',
       'perc_top10_ovr', 'perc_win_ovr'],
      dtype='object')

In [69]:
bikers_stats.columns

Index(['2015_div', '2015_fc_rank', '2015_race_days', '2015_team',
       '2015_uci_rank', '2015_wins', '2015_year_km', '2016_div',
       '2016_fc_rank', '2016_race_days', '2016_team', '2016_uci_rank',
       '2016_wins', '2016_year_km', '2017_div', '2017_fc_rank',
       '2017_race_days', '2017_team', '2017_uci_rank', '2017_wins',
       '2017_year_km', '2018_div', '2018_fc_rank', '2018_race_days',
       '2018_team', '2018_uci_rank', '2018_wins', '2018_year_km', 'followers',
       'height', 'mount_jerseys', 'nation', 'point_jerseys', 'sprint_jerseys',
       'total_wins', 'youth_jerseys', 'name'],
      dtype='object')

In [70]:
# Creating biker class dataset using all-stage biker data and counted statistics
biker_class = pd.merge(biker_by_stage, bikers_stats, left_on='name', right_on='name')
biker_class.head()

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,...,2018_wins,2018_year_km,followers,height,mount_jerseys,nation,point_jerseys,sprint_jerseys,total_wins,youth_jerseys
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
1,Y.Hutarovich,100,31,AG2R La Mondiale,10072,19455.0,2,40315.0,2014,2,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
2,Y.Hutarovich,100,31,AG2R La Mondiale,10072,13167.0,2,53482.0,2014,3,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
3,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20609.0,3,74091.0,2014,4,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0
4,Y.Hutarovich,100,31,AG2R La Mondiale,10072,17350.0,98,91441.0,2014,5,...,NaN,NaN,0,1.79,0,Belarus,6,0,8,0


In [71]:
# Rearranging columns for clearness
cols = biker_class.columns.tolist()
cols = cols[-8:] + cols[:-8]
biker_class = biker_class[cols]
biker_class = biker_class.reindex(columns=[
    'name', 'nation','age','height','team','year','stage','position','time_in_s','gc','gc_time_in_s',
    'mount_jerseys','point_jerseys','sprint_jerseys','youth_jerseys','followers','total_wins',
    'biker_sys_nr','team_sys_nr']+cols[19:])       

### Adding `stages_summary` information

In [72]:
biker_class = biker_class.merge(stages_summary,on=['year','stage'], how='inner')

### Defining `y_label` (dependend variable)

In [73]:
# Defining the label (dependent variable)
# 1 means being in Top 10 position at the given stage, 0 means being out of Top 10
biker_class['y_label'] = np.where(biker_class['position'] <= 10, 1, 0)

<br><br><br>
# Saving dataframes

### Overall stages summary: `stages_summary`

In [74]:
stages_summary.to_csv('preprocessed/pre_stages_summary.csv', index=False)
stages_summary.head()

,date,year,stage,stage_type,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility
0,2014-08-03,2014,1,flat,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9
1,2014-08-04,2014,2,flat,1,226.0,Warszawa,26.6,61,NW,4,5,7
2,2014-08-05,2014,3,flat,1,174.0,Rzeszów,22.5,65,SSE,1,4,6
3,2014-08-06,2014,4,flat,1,236.0,Katowice,22.7,52,N,3,4,8
4,2014-08-07,2014,5,hilly,2,190.0,Strbskie Pleso,25.5,66,ENE,2,1,8


### Tour results by stages: `stages`

In [75]:
stages.to_csv('preprocessed/pre_stages.csv', index=False)
stages.head(2)

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,position
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,1
1,R.Maikin,14004,24,RusVelo,3482,20870.0,2,20864.0,2014,1,2


### Biker by stage results: `biker_by_stage`

In [76]:
biker_by_stage.to_csv('preprocessed/pre_biker_by_stage.csv')
biker_by_stage.head(2)

,name,biker_sys_nr,age,team,team_sys_nr,time_in_s,gc,gc_time_in_s,year,stage,...,best_gc_ovr_l3,avg_gc_ovr_l3,worst_gc_ovr_l3,best_gc_time_ovr_l3,avg_gc_time_ovr_l3,worst_gc_time_ovr_l3,n_top10_l3,n_win_l3,perc_top10_ovr,perc_win_ovr
0,Y.Hutarovich,100,31,AG2R La Mondiale,10072,20870.0,1,20860.0,2014,1,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857
1,Y.Hutarovich,100,31,AG2R La Mondiale,10072,19455.0,2,40315.0,2014,2,...,98,109.0,117,91441.0,104600.666667,112168.0,0.0,0.0,0.571429,0.142857


### Historical bikers data: `biker_stats`

In [77]:
bikers_stats.to_csv('preprocessed/pre_biker_stats.csv')
bikers_stats.head(2)

,2015_div,2015_fc_rank,2015_race_days,2015_team,2015_uci_rank,2015_wins,2015_year_km,2016_div,2016_fc_rank,2016_race_days,...,2018_year_km,followers,height,mount_jerseys,nation,point_jerseys,sprint_jerseys,total_wins,youth_jerseys,name
0,PROF,115,70,Bretagne - Séché Environnement,-,3,11430.9,PROF,430,59,...,NaN,0,1.79,0,Belarus,6,0,8,0,Y.Hutarovich
1,PROF,301,54,RusVelo,-,1,8473.1,PROF,161,63,...,7349.7,2,NaN,0,Russia,2,0,0,0,R.Maikin


### Final biker class dataframe: `biker_class`

In [78]:
biker_class.to_csv('preprocessed/pre_biker_class.csv')
biker_class.head(2)

,name,nation,age,height,team,year,stage,position,time_in_s,gc,...,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility,y_label
0,Y.Hutarovich,Belarus,31,1.79,AG2R La Mondiale,2014,1,1,20870.0,1,...,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9,1
1,R.Maikin,Russia,24,NaN,RusVelo,2014,1,2,20870.0,2,...,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9,1


<br><br><br>
# Cleaning data

**Next notebook:** [Part 2: Data statistics and visualization](tdp_2_3_data_statistics_visualization.ipynb)

In [255]:
biker_class = pd.read_csv('preprocessed/pre_biker_class.csv', index_col=0)
biker_class.shape

(5255, 92)

## Removing columns with many missing values

In [256]:
# Removing columns with many missing values and information not needed
biker_class.drop(['height','biker_sys_nr','team_sys_nr','stage_type'], axis=1, inplace=True)
biker_class.shape

(5255, 88)

## Fixing data types & further low-information columns removal

In [257]:
# biker_class.dtypes

In [258]:
# Wrong-type columns
biker_class.loc[:,'2015_div':'2018_year_km'].dtypes

2015_div           object
2015_fc_rank       object
2015_race_days    float64
2015_team          object
2015_uci_rank      object
2015_wins         float64
2015_year_km      float64
2016_div           object
2016_fc_rank       object
2016_race_days    float64
2016_team          object
2016_uci_rank      object
2016_wins         float64
2016_year_km      float64
2017_div           object
2017_fc_rank       object
2017_race_days    float64
2017_team          object
2017_uci_rank      object
2017_wins         float64
2017_year_km      float64
2018_div           object
2018_fc_rank       object
2018_race_days    float64
2018_team          object
2018_uci_rank      object
2018_wins         float64
2018_year_km      float64
dtype: object

In [259]:
# Looking closer at the content we see there are many values as '-'
biker_class.loc[:,'2015_div':'2018_year_km'].head()

,2015_div,2015_fc_rank,2015_race_days,2015_team,2015_uci_rank,2015_wins,2015_year_km,2016_div,2016_fc_rank,2016_race_days,...,2017_uci_rank,2017_wins,2017_year_km,2018_div,2018_fc_rank,2018_race_days,2018_team,2018_uci_rank,2018_wins,2018_year_km
0,PROF,115.0,70.0,Bretagne - Séché Environnement,-,3.0,11430.9,PROF,430.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PROF,301.0,54.0,RusVelo,-,1.0,8473.1,PROF,161.0,63.0,...,-,0.0,8936.9,PROF,735.0,44.0,Gazprom - RusVelo,-,0.0,7349.7
2,PRT,640.0,73.0,Lampre - Merida,-,0.0,11960.4,PRT,375.0,70.0,...,330,0.0,8602.1,PRT,1142.0,63.0,UAE Team Emirates,295,0.0,9924.4
3,CONT,460,62.0,Optum p/b Kelly Benefit Strategies,-,1.0,8655.5,CONT,619,40.0,...,-,1.0,9546.0,PROF,423,61.0,Israel Cycling Academy,-,0.0,10098.1
4,PRT,360,72.0,Team Katusha,196,1.0,11969.9,PRT,1340,85.0,...,-,0.0,13660.8,PRT,998,17.0,Team Katusha - Alpecin,-,0.0,2733.3


In [260]:
# Replacing '-' with NaNs
biker_class.loc[:,'2015_div':'2018_year_km'] = biker_class.loc[:,'2015_div':'2018_year_km'].replace('-', np.NaN)

In [261]:
# Changing type of 'object' columns to their proper data type
rank_ix = biker_class.columns[biker_class.columns.str.contains('_rank')]
biker_class[rank_ix] = biker_class[rank_ix].astype('float')
biker_class[rank_ix].dtypes

2015_fc_rank     float64
2015_uci_rank    float64
2016_fc_rank     float64
2016_uci_rank    float64
2017_fc_rank     float64
2017_uci_rank    float64
2018_fc_rank     float64
2018_uci_rank    float64
dtype: object

In [262]:
# Checking null-values in selected columns
biker_class[rank_ix].isnull().sum()

2015_fc_rank      243
2015_uci_rank    3571
2016_fc_rank      263
2016_uci_rank    3380
2017_fc_rank      465
2017_uci_rank    2140
2018_fc_rank      823
2018_uci_rank    2541
dtype: int64

In [263]:
# Further column removal due to many missing values
biker_class.drop(list(biker_class.filter(regex='_uci_rank')), axis=1, inplace=True)
biker_class.shape

## Imputing data

In [264]:
# Checking all dataframe for missing data
biker_class_null_cols = biker_class.columns[biker_class.isnull().any()]
biker_class[biker_class_null_cols].isnull().sum()

2015_div          385
2015_fc_rank      243
2015_race_days    170
2015_team         170
2015_wins         170
2015_year_km      170
2016_div          207
2016_fc_rank      263
2016_race_days     95
2016_team          95
2016_wins          95
2016_year_km       95
2017_div          385
2017_fc_rank      465
2017_race_days    302
2017_team         302
2017_wins         302
2017_year_km      302
2018_div          666
2018_fc_rank      823
2018_race_days    541
2018_team         541
2018_wins         541
2018_year_km      541
wind_dir          291
dtype: int64

### Categorical data

In [265]:
biker_class.select_dtypes('object').isnull().sum()

name           0
nation         0
team           0
2015_div     385
2015_team    170
2016_div     207
2016_team     95
2017_div     385
2017_team    302
2018_div     666
2018_team    541
date           0
finish         0
wind_dir     291
dtype: int64

In [266]:
# Imputing missing categorical weather data with the most frequent value
biker_class['wind_dir'].fillna(biker_class['wind_dir'].value_counts().index[0], inplace=True)

In [ ]:
# Imputing missing values in '_div' and '_team' data as new category 'NONE'
biker_class.loc[:,'2015_div':'2018_year_km':3] = biker_class.loc[:,'2015_div':'2018_year_km':3].fillna('NONE')

In [276]:
# Imputing missing values in all other numerical columns
biker_class_null_cols = biker_class.columns[biker_class.isnull().any()]
biker_class[biker_class_null_cols].isnull().sum()

2015_fc_rank      243
2015_race_days    170
2015_wins         170
2015_year_km      170
2016_fc_rank      263
2016_race_days     95
2016_wins          95
2016_year_km       95
2017_fc_rank      465
2017_race_days    302
2017_wins         302
2017_year_km      302
2018_fc_rank      823
2018_race_days    541
2018_wins         541
2018_year_km      541
dtype: int64

In [ ]:
STOP - TBD

In [281]:
rank_ix = biker_class.columns[biker_class.columns.str.contains('_fc_rank')]
biker_class[rank_ix] = biker_class[rank_ix].fillna('999999')
biker_class[rank_ix].head()

,2015_fc_rank,2016_fc_rank,2017_fc_rank,2018_fc_rank
0,115,430,999999,999999
1,301,161,428,735
2,640,375,1257,1142
3,460,619,276,423
4,360,1340,765,998


In [283]:
# Imputing 0s for the rest of value-missing columns as the Biker was not riding at that period
biker_class_null_cols = biker_class.columns[biker_class.isnull().any()]
biker_class[biker_class_null_cols] = biker_class[biker_class_null_cols].fillna(0)
biker_class[biker_class_null_cols].head()

,2015_race_days,2015_wins,2015_year_km,2016_race_days,2016_wins,2016_year_km,2017_race_days,2017_wins,2017_year_km,2018_race_days,2018_wins,2018_year_km
0,70.0,3.0,11430.9,59.0,1.0,9262.8,0.0,0.0,0.0,0.0,0.0,0.0
1,54.0,1.0,8473.1,63.0,2.0,10287.0,52.0,0.0,8936.9,44.0,0.0,7349.7
2,73.0,0.0,11960.4,70.0,0.0,11537.7,55.0,0.0,8602.1,63.0,0.0,9924.4
3,62.0,1.0,8655.5,40.0,1.0,5630.7,61.0,1.0,9546.0,61.0,0.0,10098.1
4,72.0,1.0,11969.9,85.0,0.0,14288.5,85.0,0.0,13660.8,17.0,0.0,2733.3


## Saving final cleaned dataframe: `biker_class`

In [286]:
biker_class.to_csv('processed/clean_biker_class.csv')
print(biker_class.shape)
biker_class.head(2)

(5255, 84)


,name,nation,age,team,year,stage,position,time_in_s,gc,gc_time_in_s,...,stage_type_code,dist,finish,temp,rel_humid,wind_dir,wind_speed,clouds_ovrl,visibility,y_label
0,Y.Hutarovich,Belarus,31,AG2R La Mondiale,2014,1,1,20870.0,1,20860.0,...,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9,1
1,R.Maikin,Russia,24,RusVelo,2014,1,2,20870.0,2,20864.0,...,1,226.0,Bygdoszcz,31.5,38,ESE,2,6,9,1
